<a href="https://colab.research.google.com/github/kasri-mids/MovieGenreClassification/blob/master/src/Analysis/Weighted_F1_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weighted F1 Analysis

In this notebook, we calculate the weighted F1 scores from the various models we built.

In [0]:
#%tensorflow_version 2.1
!pip install tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 37kB/s 
     |████████████████████████████████| 3.9MB 35.0MB/s 
     |████████████████████████████████| 450kB 43.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=484dc13f99d7aaadfd24bdba1303505929904eec5a94851ec7bd9d51df8506fc
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc

In [0]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [0]:
# Imports

import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_hub as hub
from time import time
import io
import re
from csv import reader
from datetime import datetime
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def calculate_weighted_f1(df):
    weighted_f1 = (df.freq * df.f1score).sum()/df.freq.sum()
    df.loc[:,"wted_f1score"] = df.freq * df.f1score
    return(weighted_f1, df)

In [0]:
def calculate_weighted_f1_baseline(df):
    df.loc[:,"pred_class"] = (df.freq > 0.5).astype(int)
    df.loc[:,"tp"] = df.freq*df.pred_class
    df.loc[:,"fn"] = (1.0-df.freq)*df.pred_class
    df.loc[:,"fp"] = df.freq*(1-df.pred_class)
    df.loc[:,'macro_f1'] = (2*df.tp) / (2*df.tp + df.fp + df.fn)
    weighted_f1 = (df.freq * df.macro_f1).sum()/df.freq.sum()
    return(weighted_f1)

In [0]:
# Run 1
# Naive Classifier
prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/plots/baseline/baseline_seqlen_128_ADAM_EPOCHS_30_LR_0.0001_LOSS_macro_double_soft_f1'
train_f1score_1 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_1 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_1 = pd.read_csv(prefix + '_val.f1score')
train_wted_f1 = calculate_weighted_f1_baseline(train_f1score_1)
dev_wted_f1 = calculate_weighted_f1_baseline(dev_f1score_1)
val_wted_f1 = calculate_weighted_f1_baseline(val_f1score_1)

print(train_wted_f1, dev_wted_f1, val_wted_f1)

0.0 0.20435737206552948 0.20940170940170938


In [0]:
# Run 2
# Combined + Oversampled + Finetuning
prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/plots/baseline/baseline_seqlen_128_ADAM_EPOCHS_30_LR_0.0001_LOSS_macro_double_soft_f1'
train_f1score_2 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_2 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_2 = pd.read_csv(prefix + '_val.f1score')

train_wted_f1, train_f1score_2 = calculate_weighted_f1(train_f1score_2)
dev_wted_f1, dev_f1score_2= calculate_weighted_f1(dev_f1score_2)
val_wted_f1, val_f1score_2 = calculate_weighted_f1(val_f1score_2)

print(train_wted_f1, dev_wted_f1, val_wted_f1)

val_f1score_2[['label','wted_f1score']].sort_values(by='wted_f1score',ascending=False).head(10)

0.5539353770036566 0.4526655841560904 0.4529936486652542


,label,wted_f1score
0,Drama,0.389391
1,Comedy,0.162499
2,Romance,0.081328
4,Thriller,0.065912
6,Action,0.044921
5,Horror,0.040278
7,Crime,0.032258
3,Sci-Fi,0.022312
8,Adventure,0.017034
9,Mystery,0.013817


In [0]:
train_f1score_2.freq.values

array([0.49, 0.14, 0.27, 0.2 , 0.19, 0.3 , 0.16, 0.15, 0.24, 0.16])

In [0]:
train_f1score_1.freq.values

array([0.49, 0.14, 0.27, 0.2 , 0.19, 0.3 , 0.16, 0.15, 0.24, 0.16])

In [0]:
val_f1score_2.label.values

array(['Drama', 'Comedy', 'Romance', 'Sci-Fi', 'Thriller', 'Horror',
       'Action', 'Crime', 'Adventure', 'Mystery'], dtype=object)

In [0]:
# Run 3
# BERT+ Oversampled + Finetuning
prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlots/plots/baseline/baseline_seqlen_128_ADAM_EPOCHS_30_LR_0.0001_LOSS_macro_double_soft_f1'
train_f1score_3 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_3 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_3 = pd.read_csv(prefix + '_val.f1score')
train_wted_f1, train_f1score_3 = calculate_weighted_f1(train_f1score_3)
dev_wted_f1, dev_f1score_3 = calculate_weighted_f1(dev_f1score_3)
val_wted_f1, val_f1score_3 = calculate_weighted_f1(val_f1score_3)
print(train_wted_f1, dev_wted_f1, val_wted_f1)
val_f1score_3[['label','wted_f1score']].sort_values(by='wted_f1score',ascending=False).head(10)

0.46784244430132155 0.4259260054421544 0.4307227110822284


,label,wted_f1score
0,Drama,0.368838
1,Comedy,0.165856
2,Romance,0.078302
4,Thriller,0.058729
3,Horror,0.041568
6,Action,0.039479
9,Crime,0.022567
5,Sci-Fi,0.018035
8,Adventure,0.017400
7,Mystery,0.016214


In [0]:
# Run 4
# Poster ANN no oversampling
prefix = '/content/drive/My Drive/w266-FinalProject/Posters/plots/baseline/baseline_ADAM_EPOCHS_100_LR_0.0001_LOSS_bce_NoOS'
train_f1score_4 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_4 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_4 = pd.read_csv(prefix + '_val.f1score')

train_wted_f1 = calculate_weighted_f1(train_f1score_4)
dev_wted_f1 = calculate_weighted_f1(dev_f1score_4)
val_wted_f1 = calculate_weighted_f1(val_f1score_4)
print(train_wted_f1, dev_wted_f1, val_wted_f1)

0.23060256074159516 0.2313951532225459 0.2316597314058842


In [0]:
train_f1score_4.label.values

array(['Drama', 'Comedy', 'Thriller', 'Romance', 'Horror', 'Crime',
       'Adventure', 'Action', 'Mystery', 'Sci-Fi'], dtype=object)

In [0]:
train_f1score_4.freq.values

array([0.55, 0.34, 0.2 , 0.2 , 0.12, 0.13, 0.09, 0.15, 0.07, 0.06])

In [0]:
# Run 5
# Poster ANN with oversampling
prefix = '/content/drive/My Drive/w266-FinalProject/Posters/plots/baseline/baseline_ADAM_EPOCHS_100_LR_0.0001_LOSS_bce'
train_f1score_5 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_5 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_5 = pd.read_csv(prefix + '_val.f1score')

train_wted_f1,train_f1score_5 = calculate_weighted_f1(train_f1score_5)
dev_wted_f1,dev_f1score_5 = calculate_weighted_f1(dev_f1score_5)
val_wted_f1,val_f1score_5 = calculate_weighted_f1(val_f1score_5)
print(train_wted_f1, dev_wted_f1, val_wted_f1)
val_f1score_5[['label','f1score','wted_f1score']].sort_values(by='wted_f1score',ascending=False).head(10)

0.2085472074233628 0.20799571403353825 0.21035966121561117


,label,f1score,wted_f1score
0,Drama,0.507269,0.284071
2,Comedy,0.145086,0.049329
1,Thriller,0.145852,0.027712
3,Action,0.080914,0.012137
4,Romance,0.055814,0.011163
5,Horror,0.054916,0.006590
6,Crime,0.039844,0.005180
7,Adventure,0.033987,0.003059
8,Mystery,0.025373,0.001776
9,Sci-Fi,0.012844,0.000771


In [0]:
val_f1score_5.head()

,Unnamed: 0,id,label,threshold,freq,precision,recall,f1score,wted_f1score
0,0,0,Drama,0.5,0.56,0.563799,0.461042,0.507269,0.284071
1,1,1,Thriller,0.5,0.19,0.196319,0.116026,0.145852,0.027712
2,2,2,Comedy,0.5,0.34,0.340377,0.092191,0.145086,0.049329
3,3,3,Action,0.5,0.15,0.141069,0.056725,0.080914,0.012137
4,4,6,Romance,0.5,0.20,0.217918,0.032006,0.055814,0.011163


In [0]:
# Run 6
# BERT with oversampling and no finetuning
prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlots/plots/baseline/baseline_seqlen_128_ADAM_EPOCHS_100_LR_0.0001_LOSS_macro_double_soft_f1_NoFT'
train_f1score_6 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_6 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_6 = pd.read_csv(prefix + '_val.f1score')

train_wted_f1 = calculate_weighted_f1(train_f1score_6)
dev_wted_f1 = calculate_weighted_f1(dev_f1score_6)
val_wted_f1 = calculate_weighted_f1(val_f1score_6)
print(train_wted_f1, dev_wted_f1, val_wted_f1)

0.18409197887872247 0.2098706154739147 0.21355614998991074


In [0]:

# Run 7
# Combined with no oversampling and BERT finetuning
prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlotsAndPosters/plots/baseline/baseline_seqlen_128_ADAM_EPOCHS_30_LR_0.0001_LOSS_macro_double_soft_f1_NoOS'
train_f1score_7 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_7 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_7 = pd.read_csv(prefix + '_val.f1score')

train_wted_f1 = calculate_weighted_f1(train_f1score_7)
dev_wted_f1 = calculate_weighted_f1(dev_f1score_7)
val_wted_f1 = calculate_weighted_f1(val_f1score_7)
print(train_wted_f1, dev_wted_f1, val_wted_f1)

0.5300468833499054 0.4296415716992816 0.42926926992652464


In [0]:
# Run 8
# BERT no finetuning and no oversampling
prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlots/plots/baseline/baseline_seqlen_128_ADAM_EPOCHS_30_LR_0.0001_LOSS_macro_double_soft_f1_NoFT_NoOS'
train_f1score_8 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_8 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_8 = pd.read_csv(prefix + '_val.f1score')

train_wted_f1 = calculate_weighted_f1(train_f1score_8)
dev_wted_f1 = calculate_weighted_f1(dev_f1score_8)
val_wted_f1 = calculate_weighted_f1(val_f1score_8)
print(train_wted_f1, dev_wted_f1, val_wted_f1)

0.19098719348068774 0.1895220709899126 0.19213854562570126


In [0]:
# Run 9
# BERT with finetuning and no oversampling
prefix = '/content/drive/My Drive/w266-FinalProject/MoviePlots/plots/baseline/baseline_seqlen_128_ADAM_EPOCHS_30_LR_0.0001_LOSS_macro_double_soft_f1_NoOS'
train_f1score_9 = pd.read_csv(prefix + '_train.f1score')
dev_f1score_9 = pd.read_csv(prefix + '_dev.f1score')
val_f1score_9 = pd.read_csv(prefix + '_val.f1score')

train_wted_f1 = calculate_weighted_f1(train_f1score_9)
dev_wted_f1 = calculate_weighted_f1(dev_f1score_9)
val_wted_f1 = calculate_weighted_f1(val_f1score_9)
print(train_wted_f1, dev_wted_f1, val_wted_f1)

0.48973970860607624 0.4296917325651946 0.43337373536383805
